In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import os
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import json

In [2]:
model_path = os.environ.get("model_path", "/tmp/pipeline/model/")
etl_data_path = os.environ.get("etl_path", "/tmp/pipeline/etl/")
epoch = int(os.environ.get("epochs", "3"))
batch = int(os.environ.get("batch_size", "10"))
learning_rate = float(os.environ.get("learning_rate", "0.01"))
modelAppend = os.environ.get("model_append", "False")

In [3]:
with np.load(etl_data_path+"training_data.npz") as data:
    x_train = data['imgs']
    y_train = data['labels']

In [4]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
# (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
# x_train = x_train.astype("float32") / 255
# x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
# x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
# print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (59000, 28, 28, 1)
59000 train samples


## Build the model

In [5]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [6]:
model_file_path=model_path+"/poorna_cnn_mnist.model"
if modelAppend.lower() == "true" and os.path.exists(model_file_path) == True:
    print("going to use existing model")
    model = keras.models.load_model(model_file_path)
    model.summary()

## Train the model

In [7]:
batch_size = batch
epochs = epoch
opt = keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=opt,  # Good default optimizer to start with
              loss='sparse_categorical_crossentropy',  # how will we calculate our "error." Neural network aims to minimize loss.
              metrics=['accuracy'])  # what to track
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

checkpoint = ModelCheckpoint(filepath=model_file_path,save_best_only=True,monitor='loss',verbose=1,mode='min')

early_stop = EarlyStopping(monitor = "loss", patience = 10, verbose = 1)


model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True,verbose=1,callbacks=[early_stop, checkpoint], validation_split=0.1)

Epoch 1/3
5309/5310 [============================>.] - ETA: 0s - loss: 0.2174 - accuracy: 0.9329
Epoch 00001: loss improved from inf to 0.21734, saving model to /tmp/pipeline/model/poorna_cnn_mnist.model
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/pipeline/model/poorna_cnn_mnist.model/assets
5310/5310 [==============================] - 37s 7ms/step - loss: 0.2173 - accuracy: 0.9329 - val_loss: 0.0592 - val_accuracy: 0.9820
Epoch 2/3
5308/5310 [============================>.] - ETA: 0s - loss: 0.0928 - accuracy: 0.9716
Epoch 00002: loss improved from 0.21734 to 0.09275, saving model to /tmp/pipeline/model/poorna_cnn_mnist.model
INFO:tensorflow:Assets written to: /tmp/pipeline/model/poorna_cnn_mnist.model/assets
5310/5310 [==============================] - 33s 6

In [8]:
# model.save(model_file_path)